In [1]:
import os
import time
import chipwhisperer as cw
import subprocess
from tqdm.notebook import trange, tnrange
from setup_generic import hardware_setup, scope_reset, tracewhisperer_husky_setup
import numpy as np
import matplotlib.pyplot as plt
from sca_preprocess import trace_fft, trace_butter_lpf
from sca_preprocess import calc_snr, calc_ttest, calc_sod
from sca_preprocess import select_poi_max_rank, select_poi_threshold

TARGET = 'stm32f4'
SS_VER='SS_VER_1_1'

In [5]:
%%bash
make -B TESTFILE=fpr_scaled_clip OPT=-O0 build

SS_VER set to SS_VER_1_1
+--------------------------------------------------------
+ target: stm32f4
+ test file: fpr_smallint_and_FFT_f_one_layer
+ simple serial version: SS_VER_1_1
+ optimization level: -O3
+--------------------------------------------------------
LDSCRIPT build-stm32f4/ldscript.ld
mkdir -p build-stm32f4/obj
   CC    build-stm32f4/obj/fpr_smallint_and_FFT_f_one_layer.o fpr_smallint_and_FFT_f_one_layer.c
   CC    build-stm32f4/obj/fpr.o ./fpr_c_file/fpr.c
   CC    build-stm32f4/obj/fft.o ./fpr_c_file/fft.c
   CC    build-stm32f4/obj/simpleserial.o /home/likang/sca/cw-firmware-mcu/simpleserial/simpleserial.c
   CC    build-stm32f4/obj/tracewhisperer.o /home/likang/sca/cw-firmware-mcu/tracewhisperer/tracewhisperer.c
   CC    build-stm32f4/obj/hal.o /home/likang/sca/cw-firmware-mcu/hal/hal.c
   CC    build-stm32f4/obj/stm32f4_hal.o /home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c


/home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c: In function 'platform_init':
/home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c:87:9: note: '#pragma message: init by default settings'
   87 | #pragma message "init by default settings"
      |         ^~~~~~~
/home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c: In function 'init_uart':
/home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_hal.c:139:17: note: '#pragma message: Using 38400 baud rate for SS_VER_1_1'
  139 |         #pragma message "Using 38400 baud rate for SS_VER_1_1"
      |                 ^~~~~~~


   CC    build-stm32f4/obj/stm32f4_sysmem.o /home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_sysmem.c
   CC    build-stm32f4/obj/stm32f4xx_hal.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rng.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rng.c
   CC    build-stm32f4/obj/stm32f4xx_hal_gpio.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_gpio.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rcc.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rcc.c
   CC    build-stm32f4/obj/stm32f4xx_hal_rcc_ex.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_rcc_ex.c
   CC    build-stm32f4/obj/stm32f4xx_hal_uart.o /home/likang/sca/cw-firmware-mcu/hal/driver/stm32f4/Src/stm32f4xx_hal_uart.c
   AS    build-stm32f4/obj/stm32f4_startup.o /home/likang/sca/cw-firmware-mcu/hal/stm32f4/stm32f4_startup.S
   LD    build-stm32f4/fpr_smallint_and_FFT_f_

/opt/arm-gnu-toolchain-14.2.rel1-x86_64-arm-none-eabi/bin/../lib/gcc/arm-none-eabi/14.2.1/../../../../arm-none-eabi/bin/ld: warning: build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf has a LOAD segment with RWX permissions


Memory region         Used Size  Region Size  %age Used
             RAM:        2856 B       128 KB      2.18%
             ROM:       25528 B         1 MB      2.43%
OBJCOPY  build-stm32f4/fpr_smallint_and_FFT_f_one_layer.hex build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
OBJCOPY  build-stm32f4/fpr_smallint_and_FFT_f_one_layer.bin build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
OBJCOPY  build-stm32f4/fpr_smallint_and_FFT_f_one_layer.eep build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
OBJDUMP  build-stm32f4/fpr_smallint_and_FFT_f_one_layer.lss -h -S -z build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
   NM    build-stm32f4/fpr_smallint_and_FFT_f_one_layer.sym -n build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
Size after:
   text	   data	    bss	    dec	    hex	filename
  25508	     16	   2848	  28372	   6ed4	build-stm32f4/fpr_smallint_and_FFT_f_one_layer.elf
   text	   data	    bss	    dec	    hex	filename
      0	  25524	      0	  25524	   63b4	build-stm32f4/fpr_smalli

In [3]:
TESTFILE='fpr_scaled_clip'

In [4]:
# Common setup
PROGRAM = True
scope, target = hardware_setup(
        "build-%s/%s.hex"%(TARGET, TESTFILE), TARGET, SS_VER, PROGRAM)
scope_reset(scope, TARGET)
trace = tracewhisperer_husky_setup(scope, target)

# scope.clock
clkgen_freq0 = scope.clock.clkgen_freq  # 7363636.363636363
scope.clock.clkgen_freq = 30E6
# scope.clock.adc_mul = 4

# scope.gin
scope.gain.db = 12

# target.baud
tmp_target_baud = target.baud
target.baud = int(tmp_target_baud *
                  (scope.clock.clkgen_freq / clkgen_freq0))

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.adc.trig_count                     changed from 1718970                   to 13636153                 
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2       

In [5]:
def collect_trace_per_class(num, scope, target, trace, N=200):
    prefix = '000000000000'
    if(num>=0):
        text = (prefix+f"{num:02}"+f"{0:02}").encode('utf8')
    else:
        num = -num
        text = (prefix+f"{num:02}"+f"{1:02}").encode('utf8')
    traces = []
    for i in range(N):
        trace.arm_trace()
        scope.arm()
        target.simpleserial_write('p', text)

        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition at', i)
            continue
        # response = target.simpleserial_read('r', 1)
        # print(response)
        tmp_trace = scope.get_last_trace()
        
        # if enable TraceWhisperer
        raw = trace.read_capture_data()
        times = trace.get_rule_match_times(raw, rawtimes=False, verbose=True)
        traces.append(tmp_trace[times[0][0]*4:times[1][0]*4])

    return traces

In [8]:
# trace.set_isync_matches(addr0=0x800025e, addr1=0x8000724, match='both')
# start_idx=57
# end_idx=899

trace.set_isync_matches(addr0=0x8000a9c, addr1=0x8001020, match='both')
scope.adc.samples = 4000
traces_O0 = []
# scope.adc.timeout = 2
# pbar = tnrange(4)
# val_list=[2,4,8,16]
val_list = list(range(-31,32))
for i in tnrange(len(val_list), desc='Capturing traces'):
    num = val_list[i]
    traces = collect_trace_per_class(num, scope, target, trace, N=200)
    # print(len(traces))
    traces_O0.append(traces)
traces_O0 = np.array(traces_O0)
# np.save("fpr_norm64_O0.npy", traces_O0)
# scope.dis()
# target.dis()

Capturing traces:   0%|          | 0/63 [00:00<?, ?it/s]

      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta = 57
     899 rule # 0, delta = 842
      57 rule # 0, delta

In [10]:
np.save("./data/fpr_scaled_clip_O0_200_3368.npy", traces_O0)
scope.dis()
target.dis()